# 🔬 Research Synthetizer

**AI-powered research assistant that synthesizes comprehensive answers from multiple sources.**

<img src="img/research-synthetizer.png" alt="Overview" width="75%">


## Features
- 📂 **Local Documents**: Index and search PDFs/TXT files via FAISS vector database
- 📚 **arXiv Papers**: Automatic academic paper retrieval and analysis
- 🌐 **Web Search**: Real-time information via Tavily API (optional)
- 🤖 **Smart RAG Pipeline**: HuggingFace embeddings + cross-encoder reranking
- 💬 **Modern UI**: Gradio interface with collapsible sidebar and report management

## Tech Stack
- **LLM**: Ollama (Qwen, Mistral, Deepseek) via LangChain/LangGraph
- **Vector DB**: FAISS with `all-MiniLM-L6-v2` embeddings
- **Reranking**: Cross-encoder `ms-marco-MiniLM-L-6-v2`
- **Interface**: Gradio 6.1+

## Quick Start
1. Place documents in `./research_docs`
2. Configure `CONFIG` dictionary (model, API keys, retrieval settings)
3. Run the app - it auto-indexes documents and launches web UI
4. Ask research questions - reports saved to `./report_docs`

---
*Research tool for educational purposes. Verify important information from primary sources.*

In [ ]:
import os
import re
import string
import operator
import json
import hashlib
import pickle
import operator
from pathlib import Path
from typing import List, Dict, Any, TypedDict, Annotated, Set

In [ ]:
# Core imports (LangChain v1 / current integrations)
from langchain_ollama import ChatOllama
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.retrievers.tavily_search_api import TavilySearchAPIRetriever

# LangGraph imports
from langgraph.graph import StateGraph, END

# Reranking imports
from sentence_transformers import CrossEncoder
import numpy as np

from IPython.display import Markdown, display


In [ ]:
# =============================================================================
# Global Configuration
# =============================================================================
llms = ["qwen2.5:3b", "mistral:7b", "gpt-oss:20b"]

CONFIG: Dict[str, Any] = {
    "model_name": llms[0],
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "reranker_model": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "tavily_api_key" : None,
    "faiss_dir": "./vector_db",
    "docs_dir": "./research_docs",
    "reports_dir": "./report_docs",
    "use_reranking": True,
    "top_k_initial": 30,
    "top_k_final": 7,
    "temperature": 0.2,
    "context_window": 8192 #4096 #8192
}

In [ ]:
class ResearchState(TypedDict):
    """State for the research workflow"""
    query: str
    document_results: str
    arxiv_results: str
    web_results: str
    messages: Annotated[List, operator.add]
    final_report: str

In [ ]:
class ResearchSynthetizer:
    """Multi-source research synthetizer with RAG capabilities and persistent vector store"""
    
    def __init__(
        self,
        model_name: str = CONFIG["model_name"],
        embedding_model: str = CONFIG["embedding_model"],
        reranker_model: str = CONFIG["reranker_model"],
        tavily_api_key: str = CONFIG["tavily_api_key"],
        vector_db_path: str = CONFIG["faiss_dir"],
        docs_directory: str = CONFIG["docs_dir"],
        reports_directory: str = CONFIG["reports_dir"],
        use_reranking: bool = CONFIG["use_reranking"],
        top_k_initial: int = CONFIG["top_k_initial"],
        top_k_final: int = CONFIG["top_k_final"],
        temperature: float = CONFIG["temperature"],
        context_window: int = CONFIG["context_window"], 
    ):
        """
        Initialize the research assistant
        
        Args:
            model_name: Ollama model name (e.g., 'llama3.2', 'mistral')
            embedding_model: HuggingFace embedding model for vector search
            reranker_model: Cross-encoder model for reranking
            tavily_api_key: API key for Tavily search (optional)
            vector_db_path: Path to store the persistent vector database
            docs_directory: Directory to monitor for PDF/text files
            use_reranking: Whether to use cross-encoder reranking
            top_k_initial: Number of documents to retrieve initially
            top_k_final: Number of documents to return after reranking
        """
        # Initialize local LLM
        self.llm = ChatOllama(
            model=model_name,
            temperature=temperature,
            num_ctx=context_window
        )
        
        # Initialize embeddings for RAG using langchain-huggingface
        self.embeddings = HuggingFaceEmbeddings(
            model_name=embedding_model
        )
        
        # Initialize reranker
        self.use_reranking = use_reranking
        self.top_k_initial = top_k_initial
        self.top_k_final = top_k_final
        
        if use_reranking:
            print(f"Loading cross-encoder reranker: {reranker_model}")
            self.reranker = CrossEncoder(reranker_model)
            print("✓ Reranker loaded successfully")
        else:
            self.reranker = None
        
        # Setup paths
        self.vector_db_path = Path(vector_db_path)
        self.docs_directory = Path(docs_directory)
        self.vector_db_path.mkdir(exist_ok=True)
        self.docs_directory.mkdir(exist_ok=True)
        
        # Tracking file
        self.index_file = self.vector_db_path / "indexed_files.json"
        self.faiss_index_path = self.vector_db_path / "faiss_index"
        
        # Vector store for documents
        self.vectorstore = None
        self.documents = []
        self.indexed_files: Set[str] = self._load_indexed_files()
        
        # Initialize search APIs
        self.arxiv = ArxivAPIWrapper(top_k_results=3, doc_content_chars_max=5000)
        
        # Initialize Tavily
        self.tavily_api_key = tavily_api_key
        if tavily_api_key:
            os.environ["TAVILY_API_KEY"] = tavily_api_key
            self.tavily_retriever = TavilySearchAPIRetriever(k=3)
        else:
            self.tavily_retriever = None
        
        # Load existing vector store or initialize empty
        self._load_or_create_vectorstore()
        
        # Build the workflow graph
        self.workflow = self._build_workflow()
    
    def _get_file_hash(self, filepath: Path) -> str:
        """Generate hash of file content for change detection"""
        hasher = hashlib.md5()
        with open(filepath, 'rb') as f:
            buf = f.read()
            hasher.update(buf)
        return hasher.hexdigest()
    
    def _load_indexed_files(self) -> Set[str]:
        """Load the set of already indexed files"""
        if self.index_file.exists():
            with open(self.index_file, 'r') as f:
                data = json.load(f)
                return set(data.get('files', []))
        return set()
    
    def _save_indexed_files(self):
        """Save the set of indexed files"""
        with open(self.index_file, 'w') as f:
            json.dump({'files': list(self.indexed_files)}, f, indent=2)
    
    def _load_or_create_vectorstore(self):
        """Load existing FAISS index or create a new one"""
        if self.faiss_index_path.exists():
            try:
                print("Loading existing vector database...")
                self.vectorstore = FAISS.load_local(
                    str(self.faiss_index_path),
                    self.embeddings,
                    allow_dangerous_deserialization=True
                )
                print(f"Loaded vector database with {self.vectorstore.index.ntotal} vectors")
            except Exception as e:
                print(f"Error loading vector database: {e}")
                print("Creating new vector database...")
                self.vectorstore = None
        else:
            print("No existing vector database found. Will create on first document load.")
            self.vectorstore = None
    
    def _save_vectorstore(self):
        """Save the FAISS index to disk"""
        if self.vectorstore:
            self.vectorstore.save_local(str(self.faiss_index_path))
            print(f"Vector database saved with {self.vectorstore.index.ntotal} vectors")
    
    def scan_and_load_documents(self):
        """
        Scan the documents directory and load only new or modified files
        """
        print(f"\nScanning directory: {self.docs_directory}")
        
        # Find all PDF and text files
        file_patterns = ['*.pdf', '*.txt', '*.md']
        all_files = []
        for pattern in file_patterns:
            all_files.extend(self.docs_directory.glob(pattern))
        
        if not all_files:
            print("No PDF or text files found in the directory.")
            return
        
        print(f"Found {len(all_files)} total files")
        
        # Identify new or modified files
        new_files = []
        for file_path in all_files:
            file_hash = f"{file_path.name}:{self._get_file_hash(file_path)}"
            if file_hash not in self.indexed_files:
                new_files.append(file_path)
        
        if not new_files:
            print("No new or modified documents to process.")
            return
        
        print(f"Processing {len(new_files)} new/modified documents...")
        
        # Load new documents
        new_docs = []
        processed_hashes = []
        
        for file_path in new_files:
            try:
                if file_path.suffix.lower() == '.pdf':
                    loader = PyPDFLoader(str(file_path))
                elif file_path.suffix.lower() in ['.txt', '.md']:
                    loader = TextLoader(str(file_path))
                else:
                    continue
                
                docs = loader.load()
                new_docs.extend(docs)
                file_hash = f"{file_path.name}:{self._get_file_hash(file_path)}"
                processed_hashes.append(file_hash)
                print(f"  ✓ Loaded: {file_path.name} ({len(docs)} pages/chunks)")
            
            except Exception as e:
                print(f"  ✗ Error loading {file_path.name}: {e}")
        
        if not new_docs:
            print("No documents were successfully loaded.")
            return
        
        # Split documents
        print("\nSplitting documents into chunks...")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )
        new_chunks = text_splitter.split_documents(new_docs)
        print(f"Created {len(new_chunks)} new chunks")
        
        # Add to vector store
        if self.vectorstore is None:
            # Create new vector store
            print("Creating new vector database...")
            self.vectorstore = FAISS.from_documents(new_chunks, self.embeddings)
        else:
            # Add to existing vector store
            print("Adding to existing vector database...")
            self.vectorstore.add_documents(new_chunks)
        
        # Update indexed files
        self.indexed_files.update(processed_hashes)
        self._save_indexed_files()
        self._save_vectorstore()
        
        print(f"\n✓ Vector database updated successfully!")
        print(f"  Total vectors: {self.vectorstore.index.ntotal}")
        print(f"  Total indexed files: {len(self.indexed_files)}")
    
    def load_documents(self, file_paths: List[str]):
        """
        Legacy method for backward compatibility
        Loads specific files (use scan_and_load_documents for directory monitoring)
        """
        all_docs = []
        
        for path in file_paths:
            path_obj = Path(path)
            
            if not path_obj.exists():
                print(f"Warning: {path} does not exist, skipping...")
                continue
            
            try:
                if path_obj.suffix.lower() == '.pdf':
                    loader = PyPDFLoader(path)
                elif path_obj.suffix.lower() in ['.txt', '.md']:
                    loader = TextLoader(path)
                else:
                    print(f"Warning: Unsupported file type {path_obj.suffix}")
                    continue
                
                docs = loader.load()
                all_docs.extend(docs)
                print(f"Loaded {len(docs)} pages/chunks from {path}")
            
            except Exception as e:
                print(f"Error loading {path}: {e}")
        
        if all_docs:
            # Split documents using langchain-text-splitters
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            new_chunks = text_splitter.split_documents(all_docs)
            
            if self.vectorstore is None:
                self.vectorstore = FAISS.from_documents(new_chunks, self.embeddings)
            else:
                self.vectorstore.add_documents(new_chunks)
            
            print(f"Total chunks: {self.vectorstore.index.ntotal}")
        else:
            print("No documents were loaded successfully")
    
    def _rerank_documents(self, query: str, documents: List[Any]) -> List[Any]:
        """
        Rerank documents using cross-encoder for better relevance
        
        Args:
            query: The search query
            documents: List of documents to rerank
            
        Returns:
            Reranked list of documents
        """
        if not self.reranker or not documents:
            return documents
        
        # Prepare query-document pairs for cross-encoder
        pairs = [[query, doc.page_content] for doc in documents]
        
        # Get relevance scores from cross-encoder
        scores = self.reranker.predict(pairs)
        
        # Sort documents by score (descending)
        scored_docs = list(zip(documents, scores))
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        
        # Return top-k documents
        reranked_docs = [doc for doc, score in scored_docs[:self.top_k_final]]
        
        # Log reranking results
        print(f"\n  Reranking results:")
        for i, (doc, score) in enumerate(scored_docs[:self.top_k_final], 1):
            source = doc.metadata.get('source', 'Unknown')
            print(f"    {i}. Score: {score:.4f} - {Path(source).name}")
        
        return reranked_docs

    
        
    def _search_documents(self, state: ResearchState) -> ResearchState:
        """Search loaded documents using RAG with optional reranking"""
        query = state["query"]
        
        if not self.vectorstore:
            state["document_results"] = "No documents loaded."
            return state
        
        try:
            # Retrieve more documents initially for reranking
            k = self.top_k_initial if self.use_reranking else self.top_k_final
            docs = self.vectorstore.similarity_search(query, k=k)
            
            if not docs:
                state["document_results"] = "No relevant information found in loaded documents."
                return state
            
            print(f"  Initial retrieval: {len(docs)} documents")
            
            # Apply reranking if enabled
            if self.use_reranking:
                docs = self._rerank_documents(query, docs)
                print(f"  After reranking: {len(docs)} documents")
            
            results = []
            for i, doc in enumerate(docs, 1):
                source = doc.metadata.get('source', 'Unknown')
                page = doc.metadata.get('page', 'N/A')
                results.append(
                    f"[Document {i} - {Path(source).name}, Page {page}]\n"
                    f"{doc.page_content[:500]}..."
                )
            
            state["document_results"] = "\n\n".join(results)
            state["messages"].append(
                AIMessage(content=f"Found {len(docs)} relevant document chunks (with reranking).")
            )
        
        except Exception as e:
            state["document_results"] = f"Error searching documents: {e}"
        
        return state
    
    def _search_arxiv(self, state: ResearchState) -> ResearchState:
        """Search ArXiv for research papers"""
        query = state["query"]
        
        try:
            results = self.arxiv.run(query)
            state["arxiv_results"] = results if results else "No papers found on ArXiv."
            state["messages"].append(
                AIMessage(content="Searched ArXiv for relevant papers.")
            )
        except Exception as e:
            state["arxiv_results"] = f"Error searching ArXiv: {e}"
        
        return state
    
    def _search_web(self, state: ResearchState) -> ResearchState:
        """Search the web using Tavily"""
        if not self.tavily_retriever:
            state["web_results"] = "Tavily search not configured."
            return state
        
        query = state["query"]
        
        try:
            docs = self.tavily_retriever.invoke(query)
            if not docs:
                state["web_results"] = "No web results found."
                return state
            
            results = []
            for i, doc in enumerate(docs, 1):
                content = doc.page_content[:500]
                source = doc.metadata.get('source', 'Unknown')
                results.append(f"[Result {i} - {source}]\n{content}...")
            
            state["web_results"] = "\n\n".join(results)
            state["messages"].append(
                AIMessage(content=f"Found {len(docs)} web results.")
            )
        except Exception as e:
            state["web_results"] = f"Error searching web: {e}"
        
        return state
    
    def _synthesize_results(self, state: ResearchState) -> ResearchState:
        """Synthesize all research findings into a comprehensive report"""
        query = state["query"]
        doc_results = state.get("document_results", "")
        arxiv_results = state.get("arxiv_results", "")
        web_results = state.get("web_results", "")
        
        # Create synthesis prompt
        synthesis_prompt = f"""You are a research synthesizer. Your task is to analyze and combine information from multiple sources into a comprehensive, well-structured report.

Research Query: {query}

--- DOCUMENT SEARCH RESULTS ---
{doc_results}

--- ARXIV RESEARCH PAPERS ---
{arxiv_results}

--- WEB SEARCH RESULTS ---
{web_results}

Please synthesize the above information into a comprehensive research report that:
1. Provides a clear overview of the topic
2. Identifies key findings from each source
3. Notes any patterns, contradictions, or gaps
4. Offers actionable insights and conclusions
5. Cites sources appropriately
6. When tools return source URLs, cite them as markdown links: [Source Name](URL)
7. ONLY include links that are actually provided by the tools - never create or guess URLs

Generate your synthesis now:"""

        try:
            messages = [
                SystemMessage(content="You are an expert research analyst skilled at synthesizing information from multiple sources."),
                HumanMessage(content=synthesis_prompt)
            ]
            
            response = self.llm.invoke(messages)
            state["final_report"] = response.content
            state["messages"].append(
                AIMessage(content="Research synthesis completed.")
            )
        except Exception as e:
            state["final_report"] = f"Error during synthesis: {e}"
        
        return state
    
    def _build_workflow(self) -> StateGraph:
        """Build the research workflow graph"""
        workflow = StateGraph(ResearchState)
        
        # Add nodes
        workflow.add_node("search_documents", self._search_documents)
        workflow.add_node("search_arxiv", self._search_arxiv)
        workflow.add_node("search_web", self._search_web)
        workflow.add_node("synthesize", self._synthesize_results)
        
        # Define the flow
        workflow.set_entry_point("search_documents")
        workflow.add_edge("search_documents", "search_arxiv")
        workflow.add_edge("search_arxiv", "search_web")
        workflow.add_edge("search_web", "synthesize")
        workflow.add_edge("synthesize", END)
        
        return workflow.compile()
    
    def research(self, query: str, use_docs: bool = True, 
                 use_arxiv: bool = True, use_web: bool = False) -> str:
        """
        Conduct comprehensive research on a query
        
        Args:
            query: Research question or topic
            use_docs: Search uploaded documents
            use_arxiv: Search ArXiv papers
            use_web: Search the web (requires Tavily API key)
        
        Returns:
            Comprehensive research report
        """
        # Initialize state
        initial_state = {
            "query": query,
            "document_results": "",
            "arxiv_results": "",
            "web_results": "",
            "messages": [HumanMessage(content=f"Research query: {query}")],
            "final_report": ""
        }
        
        # Build custom workflow based on enabled sources
        workflow = StateGraph(ResearchState)
        
        nodes_to_add = []
        
        if use_docs and self.vectorstore:
            workflow.add_node("search_documents", self._search_documents)
            nodes_to_add.append("search_documents")
        
        if use_arxiv:
            workflow.add_node("search_arxiv", self._search_arxiv)
            nodes_to_add.append("search_arxiv")
        
        if use_web and self.tavily_retriever:
            workflow.add_node("search_web", self._search_web)
            nodes_to_add.append("search_web")
        
        if not nodes_to_add:
            return "No research sources enabled or available. Please enable at least one source."
        
        workflow.add_node("synthesize", self._synthesize_results)
        nodes_to_add.append("synthesize")
        
        # Connect nodes sequentially
        workflow.set_entry_point(nodes_to_add[0])
        for i in range(len(nodes_to_add) - 1):
            workflow.add_edge(nodes_to_add[i], nodes_to_add[i + 1])
        workflow.add_edge(nodes_to_add[-1], END)
        
        # Compile and run
        app = workflow.compile()
        
        print("\n" + "=" * 75)
        print("RUNNING RESEARCH WORKFLOW")
        print("=" * 75)
        
        result = app.invoke(initial_state)
        
        return result["final_report"]

## Gradio UI

In [ ]:
import gradio as gr
import string
import re
import os
from datetime import datetime

# -------------------------
# Assistant logic
# -------------------------
assistant = None

def initialize_assistant():
    global assistant
    if assistant is None:
        assistant = ResearchSynthetizer(
            model_name=CONFIG["model_name"],
            embedding_model=CONFIG["embedding_model"],
            reranker_model=CONFIG["reranker_model"],
            tavily_api_key=CONFIG["tavily_api_key"],
            vector_db_path=CONFIG["faiss_dir"],
            docs_directory=CONFIG["docs_dir"],
            reports_directory=CONFIG["reports_dir"],
            use_reranking=CONFIG["use_reranking"],
            top_k_initial=CONFIG["top_k_initial"],
            top_k_final=CONFIG["top_k_final"],
            temperature=CONFIG["temperature"],
            context_window=CONFIG["context_window"],
        )
        assistant.scan_and_load_documents()
    return assistant


# -------------------------
# Reports utilities
# -------------------------
def get_saved_reports():
    """Get saved reports and format them for display"""
    if not os.path.exists(CONFIG["reports_dir"]):
        return []
    files = sorted(
        [
            f for f in os.listdir(CONFIG["reports_dir"])
            if f.endswith(".md")
        ],
        reverse=True
    )
    # Return list of tuples: (display_name, actual_filename)
    formatted = []
    for f in files:
        # Remove .md extension and replace underscores with spaces
        display_name = f.replace(".md", "").replace("_", " ")
        # Remove timestamp pattern if present
        display_name = re.sub(r'\s+CoT\s+\d{8}\s+\d{6}$', '', display_name)
        formatted.append((display_name, f))
    return formatted


def extract_title_from_markdown(content):
    """Extract title from markdown file (first # heading)"""
    # Look for first markdown heading (# Title)
    match = re.search(r'^#\s+(.+?)$', content, re.MULTILINE)
    if match:
        return match.group(1).strip()
    
    # Fallback: try to get first non-empty line
    lines = content.strip().split('\n')
    for line in lines:
        clean_line = line.strip().lstrip('#').strip()
        if clean_line:
            return clean_line[:100]  # Limit to 100 chars
    
    return "Loaded Document"


def load_report(filename, history):
    """Load a saved report, clear chat first, extract title, and display"""
    if not filename:
        return history, "🔍 **Active Research:** *No query yet*", gr.update()

    filepath = os.path.join(CONFIG["reports_dir"], filename)
    if not os.path.exists(filepath):
        return history, "🔍 **Active Research:** *No query yet*", gr.update()

    # Read the report content
    with open(filepath, "r", encoding="utf-8") as f:
        content = f.read()

    # Extract title from markdown
    title = extract_title_from_markdown(content)
    
    # Clear history and add the loaded report
    new_history = [{"role": "assistant", "content": content}]
    
    # Update topic display
    topic_text = f"📄 **Loaded Report:** {title}"
    
    # Update radio button selection
    return new_history, topic_text, gr.update(value=filename)


# -------------------------
# Research handler
# -------------------------
def research_query(query, history):
    if not query.strip():
        return (
            history + [{"role": "assistant", "content": "Please enter a research question."}],
            "🔍 **Active Research:** *No query yet*",
            gr.update(choices=get_saved_reports()),
        )

    assistant = initialize_assistant()

    history = history + [{"role": "user", "content": query}]
    history = history + [{"role": "assistant", "content": f"🧠 **Researching:** {query}"}]

    yield (
        history,
        f"🔍 **Active Research:** {query}",
        gr.update(choices=get_saved_reports()),
    )

    result = assistant.research(
        query=query,
        use_docs=True,
        use_arxiv=True,
        use_web=True,
    )

    # Save result
    no_punct = query.translate(str.maketrans("", "", string.punctuation))
    title = re.sub(r"\s+", "_", no_punct)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{title}_CoT_{timestamp}.md"
    filepath = os.path.join(CONFIG["reports_dir"], filename)

    os.makedirs(CONFIG["reports_dir"], exist_ok=True)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(result)

    history[-1] = {"role": "assistant", "content": result}

    yield (
        history,
        f"🔍 **Active Research:** {query}",
        gr.update(choices=get_saved_reports(), value=filename),
    )


def clear_chat():
    """Clear chat and reset topic"""
    return [], "🔍 **Active Research:** *No query yet*", gr.update(value=None)


# -------------------------
# UI
# -------------------------
with gr.Blocks(title="Research Synthetizer") as demo:

    with gr.Row(elem_id="app-grid"):

        # SIDEBAR
        with gr.Column(elem_id="sidebar"):
            toggle_btn = gr.Button("◀", elem_id="sidebar-toggle")
            
            gr.Markdown("## 🔬 Research Synthetizer")

            gr.Markdown("""
            **Data Sources**
            - 📂 Local documents  
            - 📚 arXiv papers  
            - 🌐 Web sources
            """)

            clear_btn = gr.Button("🗑 Clear Chat")
            
            # SAVED REPORTS IN SIDEBAR
            gr.Markdown("---")
            gr.Markdown("### 📂 Saved Reports")
            reports_radio = gr.Radio(
                choices=get_saved_reports(),
                interactive=True,
                show_label=False,
                elem_id="reports-list",
                container=True
            )

        # MAIN
        with gr.Column(elem_id="main"):
            topic = gr.Markdown("🔍 **Active Research:** *No query yet*")
            chatbot = gr.Chatbot(elem_id="main-chatbot")
            
            with gr.Row(elem_id="input-row"):
                query = gr.Textbox(
                    placeholder="Enter your research question…",
                    show_label=False,
                    container=False,
                    scale=4
                )
                submit = gr.Button("🔍 Research", elem_id="research-btn", scale=1)

    # -------------------------
    # EVENTS
    # -------------------------
    toggle_btn.click(fn=None, inputs=None, outputs=None, js="toggleSidebar()")

    submit.click(
        research_query,
        inputs=[query, chatbot],
        outputs=[chatbot, topic, reports_radio],
    ).then(
        lambda: "",  # Clear query input after submission
        outputs=[query]
    )

    query.submit(
        research_query,
        inputs=[query, chatbot],
        outputs=[chatbot, topic, reports_radio],
    ).then(
        lambda: "",  # Clear query input after submission
        outputs=[query]
    )

    reports_radio.change(
        fn=load_report,
        inputs=[reports_radio, chatbot],
        outputs=[chatbot, topic, reports_radio],
    ).then(
        fn=None,
        inputs=None,
        outputs=None,
        js="""() => {
            setTimeout(() => {
                const chatContainer = document.querySelector('#main-chatbot');
                if (chatContainer) {
                    const scrollableDiv = chatContainer.querySelector('[class*="overflow-y-auto"]');
                    if (scrollableDiv) {
                        scrollableDiv.scrollTop = 0;
                    }
                }
            }, 200);
        }"""
    )

    clear_btn.click(
        clear_chat,
        outputs=[chatbot, topic, reports_radio]
    )

In [ ]:
# -------------------------
# Launch
# -------------------------
with open("assets/style.css", "r") as f:
    custom_css = f.read()

demo.launch(
    server_name="127.0.0.1",
    server_port=None,
    share=False,
    theme=gr.themes.Glass(),
    css=custom_css,
    js="""
    function toggleSidebar() {
        const grid = document.getElementById("app-grid");
        const btn = document.getElementById("sidebar-toggle");
        grid.classList.toggle("collapsed");
        
        if (grid.classList.contains("collapsed")) {
            btn.textContent = "▶";
        } else {
            btn.textContent = "◀";
        }
    }
    """
)